In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/i1freqdataset/I1Freq.csv.xlsx
/kaggle/input/i1freq/I1Freq.xlsx
/kaggle/input/testaa/I1Freq11.csv


# TEAM -9 WIRELESSLY UNITED 

# IMPORT IMPORTANT MODULES:

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


# LOAD AND PREPROCESS THE DATA:

In [2]:
#here we are loading the directory of dataset that we collected from pmu in simulink
data = '/kaggle/input/testaa/I1Freq11.csv'
df = pd.read_csv(data)

# Replace missing values with NaN
df['frequency'] = df['frequency'].replace('?', np.nan)
df['frequency'] = pd.to_numeric(df['frequency'], errors='coerce')

# Interpolate missing values
df['frequency'] = df['frequency'].interpolate()

# Normalize the data
scaler = MinMaxScaler()
df['frequency'] = scaler.fit_transform(df['frequency'].values.reshape(-1, 1))


# CREATE SEQUENCE

In [3]:
sequence_length = 24
data = df['frequency'].values

def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        label = data[i + sequence_length]
        sequences.append(seq)
        labels.append(label)

    return np.array(sequences), np.array(labels)


sequences, labels = create_sequences(data, sequence_length)


X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Build and train the LSTM model :

In [4]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
576/576 [==============================] - 9s 13ms/step - loss: 0.0324 - val_loss: 4.3367e-05
Epoch 2/50
576/576 [==============================] - 7s 12ms/step - loss: 6.5836e-05 - val_loss: 2.0578e-05
Epoch 3/50
576/576 [==============================] - 7s 12ms/step - loss: 3.7880e-05 - val_loss: 5.8476e-06
Epoch 4/50
576/576 [==============================] - 7s 12ms/step - loss: 2.1957e-05 - val_loss: 1.7090e-05
Epoch 5/50
576/576 [==============================] - 7s 12ms/step - loss: 2.1945e-05 - val_loss: 1.0500e-05
Epoch 6/50
576/576 [==============================] - 7s 13ms/step - loss: 2.0721e-05 - val_loss: 2.5687e-06
Epoch 7/50
576/576 [==============================] - 7s 12ms/step - loss: 1.4694e-05 - val_loss: 2.6519e-06
Epoch 8/50
576/576 [==============================] - 7s 12ms/step - loss: 1.3185e-05 - val_loss: 3.9934e-06
Epoch 9/50
576/576 [==============================] - 7s 12ms/step - loss: 1.0321e-05 - val_loss: 5.6726e-06
Epoch 10/50
576/576 [==

#  Make The Predictions

In [5]:
# Predict on the test set
predictions = model.predict(X_test)

# Inverse transform the predictions to the original scale
predictions = scaler.inverse_transform(predictions)


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(df['frequency'].iloc[-len(predictions):], predictions)
mae = mean_absolute_error(df['frequency'].iloc[-len(predictions):], predictions)
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')


144/144 [==============================] - 1s 4ms/step
Mean Absolute Error: 59.04524049724984
